In [1]:

%pwd

'C:\\Users\\godar\\Desktop\\Text-Summarize-Project\\research'

In [2]:
import os

os.chdir("../")

In [3]:
%pwd

'C:\\Users\\godar\\Desktop\\Text-Summarize-Project'

In [4]:
from dataclasses import dataclass
from pathlib import Path
# Entity(which defines the return type of function
@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path : Path
    tokenizer_name : Path

In [5]:
!pip install python-box

In [6]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config =read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self)->DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
        data_transformation_config=DataTransformationConfig(root_dir=config.root_dir,
                                                  data_path=config.data_path,
                                                  tokenizer_name=config.tokenizer_name)
        return data_transformation_config

In [8]:
import os
from src.textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset,load_from_disk

C:\Users\godar\Desktop\Text-Summarize-Project\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-03-31 13:27:48,664] : INFO : config : PyTorch version 2.6.0 available.


In [9]:
class DataTransformation:
        def __init__(self,config:DataTransformationConfig):
           print("Loaded Config:", config)
           self.config = config
           self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)

        def convert_examples_to_features(self,example_batch):
            input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

            with self.tokenizer.as_target_tokenizer():
                target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )

            return {
                'input_ids' : input_encodings['input_ids'],
                'attention_mask': input_encodings['attention_mask'],
                'labels': target_encodings['input_ids']
            }
        def convert(self):
            if not hasattr(self.config, "data_path") or not os.path.exists(self.config.data_path):
                raise ValueError(f"Invalid data path: {self.config.data_path}")
            dataset_samsum =load_from_disk(self.config.data_path)
            dataset_samsum_pt=dataset_samsum.map(self.convert_examples_to_features,batched=True)
            dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))


In [10]:
from box import ConfigBox

config_dict = {"data_path": "/path/to/data"}  # Ensure this exists
config = ConfigBox(config_dict)

print(config.data_path)  # Should not raise an error


/path/to/data


In [11]:
try:
    config=ConfigurationManager()
    print("Here..............")
    data_transformation_config=config.get_data_transformation_config()
    print("here.........")
    data_transformation=DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2025-03-31 13:27:55,434] : INFO : common : Loaded config\config.yaml successfully
[2025-03-31 13:27:55,435] : INFO : common : Loaded params.yaml successfully
[2025-03-31 13:27:55,436] : INFO : common : Created directory artifacts successfully
Here..............
[2025-03-31 13:27:55,437] : INFO : common : Created directory artifacts/data_transformation successfully
here.........
Loaded Config: DataTransformationConfig(root_dir='artifacts/data_transformation', data_path='artifacts/data_ingestion/samsum_dataset', tokenizer_name='google/pegasus-cnn_dailymail')


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]C:\Users\godar\Desktop\Text-Summarize-Project\myenv\Lib\site-packages\transformers\tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 123339.71 examples/s]
